In [3]:
with open("attention_results/20240528_115802_attention.txt", "r") as file:
    contents = file.read()
    print(contents)

Generated token index: 0, decoder element 0 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 1 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 2 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 3 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 4 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 5 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 6 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 7 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 8 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 9 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 10 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, decoder element 11 shape: torch.Size([1, 32, 769, 769])
Generated token index: 0, 

In [6]:
import json

with open("attention_results/20240528_120042_attention_weights.json", "r") as file:
    attention_description = json.load(file)

    # Remove the 'attention_description' key if it exists
    if 'attention_description' in attention_description:
        del attention_description['attention_description']

    # Display the content of the JSON file nicely
    import pprint

    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(attention_description)


{   'image_features': [1, 729, 2560],
    'image_tensor': [1, 3, 384, 384],
    'input_ids': [1, 41],
    'inputs_embeds': [1, 769, 2560],
    'outputs': 'The image captures a serene moment of two cats, one striped '
               'and the other spotted, peacefully sleeping on a pink blanket. '
               'The striped cat is curled up on the left side of the blanket, '
               'its body relaxed and at ease. On the right side of the '
               'blanket, the spotted cat is stretched out, its body stretched '
               'out in a display of tranquility. \n'
               '\n'
               'In the background, two remote controls are visible, one on '
               'each side of the image. The remote control on the left is '
               'blue, while the one on the right is white. These objects add a '
               'touch of human presence to the scene, suggesting that this '
               'peaceful moment might be taking place in a home setting. \n'
         